<a href="https://colab.research.google.com/github/HandanYU/Rumour-detection/blob/main/Pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- replace_abbreviations
- remove url 
- get the emoji and replace them as words
- remove stop words
- keep only english letters
- stemming

In [ ]:
# !pip install emoji
# !pip install nltk
# !pip install textblob

  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=5a92f91dcf134fc24011694b0e97f509edb85899e2f6e4aeb666514621a3bb94
  Stored in directory: c:\users\trist\appdata\local\pip\cache\wheels\5e\8c\80\c3646df8201ba6f5070297fe3779a4b70265d0bfd961c15302
Successfully built emoji


### Sort raw data by time

In [ ]:
@timer('ms')

def sort_by_time(raw_file, json_file):
    with open(raw_file) as file:
        ids = file.readlines()

    with open(json_file, 'r+') as file:
        content = file.read()
        content=json.loads(content)
        df = pd.DataFrame(content)
        df = df.T

    save_name = raw_file[:-4] + '_sorted.txt'
    with open(save_name, 'w') as file:
        date = pd.Series(pd.DatetimeIndex(df.iloc[:, 6]), index=df.index)
        df.drop(['created_at'], axis=1, inplace=True)
        df['time'] = date

        for id_ in ids:
            ids_ = id_.strip().split(',')
            source_id = ids_[0]
            file.write(source_id)
            if len(ids_) > 1:
                reply_ids = ids_[1:]
                reply_ids[-1] = reply_ids[-1].replace('\n', '')
                valid_ids = [index for index in reply_ids if index in df.index]
                sorted_replies = df.loc[valid_ids].sort_values(by='time')
                if len(valid_ids) > 0:
                    file.write(',')

                for i, index in enumerate(sorted_replies.index):
                    file.write(index)
                    if i != len(sorted_replies.index) - 1:
                        file.write(',')

            file.write('\n')

In [ ]:
raw_files = ['train.data.txt', 'dev.data.txt']
json_files = ['./filtered data/train_reply.json', './filtered data/dev_reply.json']
for raw_file, json_file in zip(raw_files, json_files):
    sort_by_time(raw_file, json_file)

In [ ]:
with open(raw_files[0]) as file:
    train = file.readlines()
with open('train.data_sorted.txt') as file:
    s = file.readlines()
assert len(train) == len(s)

## Text pre-processing

In [1]:
import emoji
from nltk.corpus import stopwords
import re
import nltk
import json
import pandas as pd
from utils import timer
from datetime import datetime
from textblob import TextBlob
# from nltk.corpus import wordnet

/Users/yuhandan/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.8' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
nltk.download('wordnet')
stemmer = nltk.stem.porter.PorterStemmer()
stopword = stopwords.words('english')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/yuhandan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:



@timer('ms')
# def replace_duplicate_letter(word):
#     repeat_reg = re.compile(r'(\w*)(\w)\2(\w*)')
#     repl = r'\1\2\3'
#     if wordnet.synsets(word):  # 判断当前字符串是否是单词
#         return word
#     repl_word = repeat_reg.sub(repl, word)
#     if repl_word != word:
#         return replace_duplicate_letter(repl_word)
#     else:
#         return repl_word
  
def clean_tweet(content):
    from time import strftime
    # def compute_num_month(content):
    #     month_num = 0
    #     month = ["january", "february", "march", "april", "may", "june", "july", "august", "september", "octorber", "november", "december", 
    #             "jan.", "feb.", "mar.", "apr.", "may.", "jun.", "jul.", "aug.", "sept.", "oct.", "nov.", "dec."]
    #     for i in content:
    #         if i in month:
    #             month_num += 1
    #     return month_num
    # replace_abbreviations
    content = content.lower()
    content = re.sub(r"won't", "will not", content)
    content = re.sub(r"can't", "can not", content)
    content = re.sub(r"cannot", "can not", content)
    content = re.sub(r"n't", " not", content)
    content = re.sub(r"'re", " are", content)
    content = re.sub(r"'s", " is", content)
    content = re.sub(r"'d", " would", content)
    content = re.sub(r"'ll", " will", content)
    content = re.sub(r"'t", " not", content)
    content = re.sub(r"'ve", " have", content)
    content = re.sub(r"'m", " am", content)
    content = re.sub(r".”", " ", content)
    
    # get the number of month be mentioned
    # month_num = compute_num_month(content)

    # get the number of url
    mentioned_url_num = len(re.findall(r'https?://[^ ]+', content))
    mentioned_url_num += len(re.findall(r'www.[^ ]+', content))
    # get the number of twitter ID be mentioned
    id_num = len(re.findall(r'@[A-Za-z0-9_]+', content))
    content = re.sub(r'@[A-Za-z0-9_]+', '', content) # remove twitter ID
    # remove url 
    ## http, https
    content = re.sub(r'https?://[^ ]+', '', content) 
    ## www.
    content = re.sub(r'www.[^ ]+', '', content)
    # get the emoji and replace them as words
    emojis = emoji.distinct_emoji_list(content)
    for e in emojis:
        content = re.sub(e, emoji.demojize(e), content)
    content = re.sub('\w+\d+\w+', '', content) # remove the word contains numbers
    
    content = re.sub(r'[:_!+“\-=——,$%^\?\\~\"\'@#$%&*<>{}\[\]()/]', ' ', content) # remove punctuation, except .
    
    content = re.sub(r"\s+", " ", content) # conver multiple spaces as a single space
    content = content.strip()
    
    # remove stop words 
    # TODO: and keep only english letters
    
    content = [c for c in content.split(' ') if c not in stopword and c.isalpha()]
    # do stemming
    content = [stemmer.stem(token.strip()) for token in content]
    
    return ' '.join(content), mentioned_url_num, id_num 
    #, month_num

In [4]:
# @timer('ms')
def json2df(json_file):
    """
    json_file: 'train_reply.json'
    """
    with open(json_file,'r+') as file:
        content = file.read()
    content = json.loads(content)
    df = pd.DataFrame(content)
    df = df.T
    return df
# df = json2df('train_reply.json')
def clean_data(data_type):
  """
  Args: 
    data_type: 'dev', 'train'
  Returns:
    source_df, reply_df
  """
  source_df = json2df(f'./data/full data/{data_type}_source.json')
  source_df['temp'] = source_df['text'].apply(lambda x: clean_tweet(x))
  source_df['text'] = source_df['temp'].apply(lambda x: x[0])
  source_df['mentioned_url_num'] = source_df['temp'].apply(lambda x: x[1])
  source_df['id_num'] = source_df['temp'].apply(lambda x: x[2])
  source_df = source_df.drop(columns='temp')
  source_df['tweet_id'] = source_df.index

  reply_df = json2df(f'./data/full data/{data_type}_reply.json')
  reply_df['temp'] = reply_df['text'].apply(lambda x: clean_tweet(x))
  reply_df['text'] = reply_df['temp'].apply(lambda x: x[0])
  reply_df['mentioned_url_num'] = reply_df['temp'].apply(lambda x: x[1])
  reply_df['id_num'] = reply_df['temp'].apply(lambda x: x[2])
  reply_df = reply_df.drop(columns='temp')
  reply_df['tweet_id'] = reply_df.index
  
  return source_df, reply_df
# train_source_df, train_reply_df = clean_data('train')

# dev_source_df, dev_reply_df = clean_data('dev')


## Concat label

In [5]:
import pandas as pd
def concat_label(data_type, source_feature_df):
    """Concat labels on source tweets
    data_type: 'dev', 'train'
    """
    df = pd.DataFrame(columns=['tweet_id', 'label'])
    with open(f'./data/original_data/{data_type}_source.txt', 'r') as f:
        ids = f.readlines()
    with open(f'./data/original_data/{data_type}.label.txt', 'r') as f:
        labels = f.readlines()
    df['tweet_id'] = [id.strip() for id in ids]
    df['label'] = [label.strip() for label in labels]
    df_labels = pd.merge(source_feature_df, df, on='tweet_id', how='left')
    df_labels['label'] = df_labels['label'].apply(lambda x: 0 if x == 'nonrumour' else 1)
    return df_labels

# dev_source_df = concat_label('dev', dev_source_df)
# train_source_df = concat_label('train', train_source_df)


## Concat Sorted reply

In [6]:
def concat_reply(data_type, source_df):
    """concat replies on source tweets
    data_type: 'dev', 'train'
    """
    df = pd.DataFrame(columns=['tweet_id', 'reply'])
    with open(f'./data/original_data/{data_type}.data_sorted.txt', 'r') as f:
        content = f.readlines()
    df['tweet_id'] = [c.split(',')[0].strip() for c in content]
    df['reply'] = [','.join([i.strip() for i in c.split(',')[1:]]) for c in content]
    source_df = pd.merge(source_df, df, on='tweet_id', how='left')
    return source_df

# dev_source_df = concat_reply('dev', dev_source_df)
# train_source_df = concat_reply('train', train_source_df)


## check whether the created date is on weekday

In [7]:
def check_weekday(df):
    """
    df: source_df or reply_df
    """
    df['isoweekday'] = df['created_at'].apply(lambda x: datetime.strptime(x.split('T')[0], '%Y-%m-%d').isoweekday())
    df['isweekday'] = df['isoweekday'].apply(lambda x: 1 <= x <= 5)
    df = df.drop(columns='isoweekday')
    return df

## Concat user info

In [8]:
def concat_user_info(data_type, source_df):
    """concat user info on source tweets
    data_type: 'dev', 'train'
    """
    df = json2df(f'./data/full data/{data_type}_source_userinfo.json')
    df['user_id'] = df.index
    source_df = pd.merge(source_df, df, on='user_id', how='left')
    return source_df



## Concat reply's features

In [ ]:
def concat_reply_info(reply_ids, reply_df, statis_feature):
    statistic_dict = dict()   
    replies_txt = ''
    if reply_ids == '':
        return [None] * (len(statis_feature) + 1)
    for r in reply_ids.split(','):
        # reply_num = len(reply_ids.split(','))
        replies_txt += reply_df.loc[r]['text']
        ## get statistic features
        for f in statis_feature:
            statistic_dict[f] = statistic_dict.get(f, 0) + int(reply_df.loc[r][f])    
    res_values = [replies_txt]
    for f in statis_feature:
        res_values.append(statistic_dict[f])
    return res_values

# Processing

In [59]:
def processing(data_type):
    
    source_df, reply_df = clean_data(data_type)
    source_df = concat_label(data_type, source_df)
    source_df = concat_reply(data_type, source_df)
    source_df = check_weekday(source_df)
    source_df = concat_user_info(data_type, source_df)
    reply_df = check_weekday(reply_df)
    # add sentiment score
    source_df['senti_score'] = source_df['text'].apply(lambda x: 1 if TextBlob(x).sentiment.polarity > 0 else 0)
    reply_df['senti_score'] = reply_df['text'].apply(lambda x: 1 if TextBlob(x).sentiment.polarity > 0 else 0)
    reply_df.index = reply_df['tweet_id']
    # concat replies info to source_df
    statis_feature = ['reply_count', 'like_count', 'retweet_count', 'quote_count', 
                        'possibly_sensitive', 'has_url', 'mentioned_url_num', 
                        'id_num', 'isweekday', 'senti_score']
    source_df[['reply_text'] + statis_feature] = source_df.apply(lambda x: concat_reply_info(x['reply'], reply_df, statis_feature), axis=1, result_type='expand')                

    return source_df

In [ ]:
train_df = processing('train')
dev_df = processing('dev')

In [ ]:
train_source_df.to_csv('./data/filtered_data/train_source_df.csv')
train_reply_df.to_csv('./data/filtered_data/train_reply_df.csv')
dev_source_df.to_csv('./data/filtered_data/dev_source_df.csv')
dev_reply_df.to_csv('./data/filtered_data/dev_reply_df.csv')


In [57]:
source_df

,text,reply_count,like_count,retweet_count,quote_count,possibly_sensitive,created_at,user_id,has_url,mentioned_url_num,id_num,tweet_id,label,reply,isweekday,followers_count,tweet_count,senti_score,reply_text
0,covid fact hand dryer effect kill new coronavi...,0.0,0.0,0.0,0.0,0.0,2020-04-11T16:01:39.000Z,35761403,1.0,0.0,1.0,1249004694950817796,0,1249011200068730880,0.0,34984,64281,0.0,fact germ breed
1,expect result husband pend antibodi test pleas...,2.0,0.0,0.0,0.0,0.0,2020-06-01T20:23:06.000Z,2734457193,2.0,0.0,4.0,1267552274819227649,0,"1270394169836568576,1270502071175909376",2.0,758,14269,0.0,hi luck boat
2,covid spread peopl catch covid other diseas sp...,9.0,3.0,2.0,1.0,0.0,2020-03-04T16:19:03.000Z,53980699,3.0,0.0,0.0,1235238334722699265,0,"1235234904281165825,1235234927937048577,123523...",10.0,3010,111037,3.0,read lot corona viru late think share highligh...
3,everi news outlet use headlin like antibiot ef...,0.0,1.0,0.0,0.0,0.0,2020-04-10T22:56:50.000Z,393187879,1.0,0.0,1.0,1248746792914546688,0,1248775858120097792,0.0,939,17292,0.0,appar headlin question answer usual
4,research encount goliath birdeat world largest...,5.0,2.0,1.0,0.0,0.0,2014-10-19T12:59:47.000Z,39585367,26.0,10.0,59.0,523820806917603328,0,"523821510042353664,523822210071658496,52382235...",4.0,1353894,46894,4.0,eu tenho uma dessa em casa são ótima para coça...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
533,cure covid howev sever ongo clinic trial inclu...,6.0,0.0,0.0,0.0,0.0,2020-04-13T06:17:21.000Z,817903682847182848,0.0,0.0,0.0,1249582429565829120,0,"1249582412004306944,1249582416827727872,124958...",7.0,113,2169,1.0,treatment option covid includ drug vaccin ther...
534,specul arrest banksi debut new mural bristol,1.0,0.0,0.0,0.0,0.0,2014-10-22T11:15:21.000Z,20622594,2.0,0.0,3.0,524881688825167872,1,"524882269941809153,525128786074173440",2.0,1971453,58412,0.0,stori say someon vandalis black
535,question answer red question mark repli number...,1.0,0.0,0.0,0.0,0.0,2020-03-20T07:51:15.000Z,570876529,0.0,0.0,0.0,1240908749256232960,0,"1240908751063965696,1240908752842362881",2.0,158,736,0.0,catch covid infect surfac packag infect area p...
536,anonym oper kkk klux klan never stop watch opk...,0.0,0.0,0.0,0.0,0.0,2015-11-02T08:49:29.000Z,247399641,2.0,1.0,4.0,661102820976930816,1,"661241325237444608,661250292550270976",2.0,103802,231196,0.0,call men hide time truth come
